In [59]:

import pandas as pd
import numpy as np
import plotly.express as px
import itertools
import subprocess
import re
import os
from tqdm import tqdm

DOCKER_IMAGE = "oqs-curl"
RESULTS_FILE = "results_{}.csv"


In [15]:
algorithms = pd.read_csv("algorithms.csv")
kem_algorithms = algorithms[algorithms["function"] == "kem"]
sig_algorithms = algorithms[algorithms["function"] == "sig"]

pairings = list(itertools.product(kem_algorithms["name"], sig_algorithms["name"]))
print(f"Loaded {len(kem_algorithms)} kem algorithms and {len(sig_algorithms)} sig algorithms, {len(pairings)} pairings")

Loaded 6 kem algorithms and 9 sig algorithms, 54 pairings


In [16]:
def parse_output(output):
    rows = []
    for line in output.split("\n"):
        line = line.strip().split(":")
        if line[0] == "+DTP":
            alg = line[1].lower()
            op = line[2]
        elif line[0].startswith("+R"):
            sec = float(line[3])
            ops = int(line[1])
            rows.append([alg, op, sec, ops])
    df = pd.DataFrame(rows, columns=["alg", "op", "sec", "ops"])
    df["ops_per_sec"] = df["ops"] / df["sec"]
    return df

In [49]:
def start_server(kem, sig):
    kill_container("pqc-test-server")
    command =f"docker run -d --rm --name pqc-test-server -e KEM_ALG={kem} -e SIG_ALG={sig} {DOCKER_IMAGE} openssl s_server -cert /opt/test/server.crt -key /opt/test/server.key -groups {kem} -www -tls1_3 -accept :4433&"
    result = subprocess.check_output(command, shell=True, text=True, stderr=subprocess.STDOUT)
    print(f"Server container ID: {result}")
    return result

def kill_container(container_name):
    command = f"docker kill {container_name}"
    killed = subprocess.call(command, shell=True, text=True, stderr=subprocess.DEVNULL)
    print(f"Container {container_name}: {'Killed' if killed == 0 else 'Already dead'}")

In [72]:
def run_test(kem, sig, test_time=10):
    try:
        start_server(kem, sig)
        command = f"docker run --rm --name pqc-test-client --link pqc-test-server -e KEM_ALG={kem} -e SIG_ALG={sig} {DOCKER_IMAGE} openssl s_time -connect pqc-test-server:4433 -new -time {test_time} -verify 1 | grep connections"
        result = subprocess.check_output(command, shell=True, text=True, stderr=subprocess.STDOUT)
        match = re.search(r"; (\d+\.\d+) connections/user sec", result)
        if not match:
            raise ValueError(f"No match found in result for KEM {kem} with SIG {sig}")
        connections_per_sec = float(match.group(1))
        print(f"KEM {kem} with SIG {sig} is {connections_per_sec} connections/user sec")
        return connections_per_sec
    finally:
        kill_container("pqc-test-client")
        kill_container("pqc-test-server")

In [73]:
PLATFORM = "i7"
if not os.path.exists(RESULTS_FILE.format(PLATFORM)):
    results = pd.DataFrame(columns=["kem", "sig", "ops_per_sec"])
else:
    results = pd.read_csv(RESULTS_FILE.format(PLATFORM))
for pairing in tqdm(pairings):
    if not results.loc[(results["kem"] == pairing[0]) & (results["sig"] == pairing[1])].empty:
        print(f"Skipping {pairing} because it already exists")
        continue
    connections_per_sec = run_test(*pairing)
    new_row = pd.DataFrame([{"kem": pairing[0], "sig": pairing[1], "ops_per_sec": connections_per_sec}])
    results = pd.concat([results, new_row], ignore_index=True)
    results.to_csv(RESULTS_FILE.format(PLATFORM), index=False)

Skipping ('kyber512', 'falcon512') because it already exists
Error response from daemon: cannot kill container: pqc-test-server: No such container: pqc-test-server
Container pqc-test-server: Already dead
Server container ID: ad11bb3a6f04bf57c0eb0f4e0adb00ec2c3a763a7602d5a7fe553a897c4969d7

KEM kyber512 with SIG falcon1024 is 2703.48 connections/user sec
Error response from daemon: cannot kill container: pqc-test-client: No such container: pqc-test-client
Container pqc-test-client: Already dead
pqc-test-server
Container pqc-test-server: Killed
Error response from daemon: cannot kill container: pqc-test-server: No such container: pqc-test-server
Container pqc-test-server: Already dead
Server container ID: 9dbd6f770474f25619693138c0dc0b6bc2090d17e8f0fbd1950efb219f370657

KEM kyber512 with SIG p256_falcon512 is 2723.11 connections/user sec
Error response from daemon: cannot kill container: pqc-test-client: No such container: pqc-test-client
Container pqc-test-client: Already dead
pqc-test-

KeyboardInterrupt: 

In [74]:
results = []
for platform in ["i7", "rpi3b"]:
    platform_results = pd.read_csv(RESULTS_FILE.format(platform))
    platform_results["platform"] = platform
    results.append(platform_results)
results = pd.concat(results)
results

FileNotFoundError: [Errno 2] No such file or directory: 'results_rpi3b.csv'

In [1]:

KEM_ALGS = ["kyber512", "kyber768", "kyber1024", "x25519_kyber512", "x25519_kyber768", "X25519"]
SIG_ALGS = ["falcon512", "falcon1024", "p256_falcon512", "p512_falcon1024", "dilithium2", "dilithium3", "p256_dilithium2", "p384_dilithium3", "ED25519"]
pairings = itertools.product(KEM_ALGS, SIG_ALGS)

recordings = []

for kem, sig in pairings:
    if os.path.exists(f"results_{kem}_{sig}.txt"):
        print(f"{kem} with {sig} already exists")
        result = open(f"results_{kem}_{sig}.txt").read()
    else:
        print(f"Running {kem} with {sig}")
        command = f"docker run -e TEST_TIME=8 -e KEM_ALG={kem} -e SIG_ALG={sig} oqs-curl perftest.sh"
        result = subprocess.check_output(command, shell=True, text=True)
        with open(f"results_{kem}_{sig}.txt", "w") as f:
            f.write(result)
    match = re.search(r"; (\d+\.\d+) connections/user sec", result)
    if match:
        pairing_performance = float(match.group(1))
        print(f"{kem} with {sig} is {pairing_performance}")
        recordings.append({
            "kem": kem,
            "sig": sig,
            "ops_per_sec": pairing_performance
        })

recordings = pd.DataFrame(recordings)
recordings["kem"] = recordings["kem"].str.lower()
recordings["sig"] = recordings["sig"].str.lower()

NameError: name 'itertools' is not defined

In [4]:
import random


recordings["sig_type"] = recordings["sig"].apply(lambda x: "hybrid" if "_" in x else "pqc" if "dilithium" in x else "trad")
recordings["kem_type"] = recordings["kem"].apply(lambda x: "hybrid" if "_" in x else "pqc" if "kyber" in x else "trad")

recordings["is_hybrid"] = (recordings["sig_type"] == "hybrid") | (recordings["kem_type"] == "hybrid")
recordings["pure_pqc"] = (recordings["sig_type"] == "pqc") & (recordings["kem_type"] == "pqc")

recordings["type"] = np.where(recordings["is_hybrid"], "Hybrid", np.where(recordings["pure_pqc"], "Pure PQ", "Traditional"))

recordings["name"] = recordings.apply(lambda row: f"{row['sig']} / {row['kem']}", axis=1)

recordings["platform"] = "i7"
rpi3b = recordings.copy()
rpi3b["ops_per_sec"] = rpi3b["ops_per_sec"].map(lambda x: x / (20 + 0 * random.random()))
rpi3b["platform"] = "rpi3b"
both = pd.concat([recordings, rpi3b])
# filtered = recordings[recordings.apply(lambda row: (row["kem"], row["sig"]) in small, axis=1)]
both.sort_values(by="ops_per_sec", ascending=False, inplace=True)
# filtered = recordings[recordings["sig"] == "dilithium2"]

fig = px.bar(both[both["platform"] == "i7"],
             x="name", 
             y="ops_per_sec", 
             color="type", 
             labels={"ops_per_sec": "Handshakes/s (higher is better)"},
             text="name",
)
fig.update_layout(showlegend=True)
fig.update_yaxes(matches=None)
fig.update_xaxes(matches=None, title="KEM / Signature Algorithms")
fig.update_layout(legend_title_text='Algorithm Type')
fig.update_layout(
    autosize=False,
    width=1600,
    height=800,
    xaxis={"categoryorder": "total descending"},
    margin={"l": 0, "r": 0, "t": 0, "b": 0}
)
fig.show()
fig.write_image("handshake_perf_i7.png")

In [51]:
import os

platforms = ["rpi3b", "i7"]

all_results = pd.DataFrame()
for platform in platforms:
    if not os.path.exists(f"results_{platform}.txt"):
        continue
    with open(f"results_{platform}.txt") as f:
        output = f.read()
    df = parse_output(output)
    df["platform"] = platform
    all_results = pd.concat([all_results, df])

all_results

,alg,op,sec,ops,ops_per_sec,platform
0,x25519,keygen,9.94,20086,2020.724346,rpi3b
1,x25519,encaps,10.00,5452,545.200000,rpi3b
2,x25519,decaps,10.00,7734,773.400000,rpi3b
3,kyber512,keygen,9.99,14291,1430.530531,rpi3b
4,kyber512,encaps,9.99,11280,1129.129129,rpi3b
5,kyber512,decaps,10.00,11330,1133.000000,rpi3b
6,x25519_kyber512,keygen,10.00,8193,819.300000,rpi3b
7,x25519_kyber512,encaps,10.00,3755,375.500000,rpi3b
8,x25519_kyber512,decaps,10.00,3775,377.500000,rpi3b
9,kyber768,keygen,9.99,8979,898.798799,rpi3b


In [56]:
import plotly.express as px
from plotly.subplots import make_subplots
# Filter the data for the required groups and operations

all_results["type"] = np.where(
    all_results["alg"].str.contains("_"), 
    "hybrid", 
    np.where(
        all_results["alg"].isin(["kyber512", "kyber768", "kyber1024", "dilithium2", "dilithium3"]), 
        "pqc", 
        "trad"
    )
)

def key_size(row):
    if "kyber" in row["alg"]:
        return int(row["alg"].split("kyber")[1])
    elif "dilithium" in row["alg"]:
        return int(row["alg"].split("dilithium")[1]) * 256
    elif "x25519" in row["alg"]:
        return 256
    elif "p256" in row["alg"]:
        return 256
    elif "p384" in row["alg"]:
        return 384
    elif "ECDSA-SHA2-256" in row["alg"]:
        return 256
    elif "ECDSA-SHA2-384" in row["alg"]:
        return 384
    else:
        return None

all_results["key_size"] = all_results.apply(key_size, axis=1)

# print(all_results)
# all_results["key_size"] = all_results.apply(key_size, axis=1)
def compare_algs(algs, platform):
    filtered_df = all_results[
        (all_results["alg"].isin(algs))
        # (all_results["platform"] == platform)
    ]
    # fig = px.bar(
    #     filtered_df, 
    #     x="op", 
    #     y="ops_per_sec", 
    #     color="alg", 
    #     barmode="stack", 
    #     labels={"ops_per_sec": "ops/sec (higher is better)"},
    #     facet_row="platform",
    #     facet_col="alg",
    #     text="op"
    # )
    # fig = px.bar(
    #     filtered_df, 
    #     x="op", 
    #     y="ops_per_sec", 
    #     color="type", 
    #     barmode="group", 
    #     labels={"ops_per_sec": "ops/sec (higher is better)", "op": "Operation", "platform": "Platform"},
    #     facet_row="platform",
    #     # facet_col="key_size",
    #     text="type"
    # )
    fig = px.bar(
        filtered_df, 
        x="type", 
        y="ops_per_sec", 
        color="op", 
        barmode="stack", 
        labels={"ops_per_sec": "Operations/sec (higher is better)", "op": "Operation", "platform": "Platform"},
        facet_row="platform",
        # facet_col="key_size",
        text="op"
    )
    fig.update_layout(showlegend=False)
    fig.update_yaxes(matches=None)
    fig.update_layout(
        autosize=False,
        width=1200,
        height=800,
    )
    fig.show()
    fig.write_image(f"ops_per_sec.png")
compare_algs(["x25519","x25519_kyber512", "kyber512"], "rpi3b")
print(all_results)

                alg      op    sec      ops    ops_per_sec platform    type  \
0            x25519  keygen   9.94    20086    2020.724346    rpi3b    trad   
1            x25519  encaps  10.00     5452     545.200000    rpi3b    trad   
2            x25519  decaps  10.00     7734     773.400000    rpi3b    trad   
3          kyber512  keygen   9.99    14291    1430.530531    rpi3b     pqc   
4          kyber512  encaps   9.99    11280    1129.129129    rpi3b     pqc   
5          kyber512  decaps  10.00    11330    1133.000000    rpi3b     pqc   
6   x25519_kyber512  keygen  10.00     8193     819.300000    rpi3b  hybrid   
7   x25519_kyber512  encaps  10.00     3755     375.500000    rpi3b  hybrid   
8   x25519_kyber512  decaps  10.00     3775     377.500000    rpi3b  hybrid   
9          kyber768  keygen   9.99     8979     898.798799    rpi3b     pqc   
10         kyber768  encaps   9.99     7104     711.111111    rpi3b     pqc   
11         kyber768  decaps  10.00     7177     717.

In [71]:
lines = output.split("\n")
rows = [line.split(":") for line in lines if line.startswith("+R")]
rows = [[row[2], float(row[3]), int(row[1])] for row in rows]
df = pd.DataFrame(rows, columns=["alg", "sec", "ops"])
df["ops_per_sec"] = df["ops"] / df["sec"]
df

,alg,sec,ops,ops_per_sec
0,X25519,9.98,448872,44977.154309
1,X25519,9.98,188835,18921.342685
2,X25519,10.00,365051,36505.100000
3,kyber512,9.83,1318721,134152.695829
4,kyber512,9.91,1159403,116993.239152
5,kyber512,9.99,1577365,157894.394394
6,x25519_kyber512,9.94,296202,29798.993964
7,x25519_kyber512,9.97,165941,16644.032096
8,x25519_kyber512,10.00,181858,18185.800000
9,kyber768,9.90,877995,88686.363636


In [1]:
def dot_product(u: list[int], v: list[int]) -> int:
    if len(u) != len(v):
        raise ValueError("Vectors must be of the same length")
    return sum(u[i] * v[i] for i in range(len(u)))

dot_product([1, 2, 3], [4, 5, 6])


32